# Data Collection and Processing

## 1. Dependencies

In [1]:
import pandas as pd
import yfinance as yf
import wikipedia as wp
import numpy as np
import dcor
import networkx as nx

## 2. Import S&P 100 List and Information

**List of S&P 100 Companies**

In [2]:
url=wp.page("S&P 100").html().encode("UTF-8")
stocks=pd.read_html(url)[2].set_index("Symbol")
stocks=stocks.drop("GOOG")
stocks.loc["GOOGL","Name"]="Alphabet"
stocks=stocks.rename(index={"BRK.B":"BRK-B"})
stocks.head()

,Name,Sector
Symbol,,
AAPL,Apple Inc.,Information Technology
ABBV,AbbVie,Health Care
ABT,Abbott Laboratories,Health Care
ACN,Accenture,Information Technology
ADBE,Adobe Inc.,Information Technology


In [3]:
stocks.to_csv("../data/stocks.csv")

**Fundamental Data of companies**

In [4]:
fundamentals=[yf.Ticker(stock).info for stock in stocks.index]

fundamentals=pd.DataFrame(fundamentals).set_index("symbol")
fundamentals.index = fundamentals.index.rename("Symbol")
fundamentals= fundamentals[["marketCap", "beta", "52WeekChange", "averageVolume", "profitMargins"]].fillna(0)
fundamentals.head()

,marketCap,beta,52WeekChange,averageVolume,profitMargins
Symbol,,,,,
AAPL,3797088731136,1.240,0.376805,43955101,0.23971
ABBV,310855827456,0.613,0.113816,6216840,0.09223
ABT,195785850880,0.722,0.046791,5046883,0.13992
ACN,218833551360,1.245,0.026692,2437208,0.11194
ADBE,193621966848,1.299,-0.230300,3459643,0.25588


**Log Daily Return**

In [5]:
values=[yf.Ticker(stock).history(period="5y",actions=False) for stock in stocks.index]

for idx, stock_values in enumerate (values):
    stock_values["NormClose"]=(stock_values["Close"].mean())/stock_values["Close"].std()
    
    stock_values["DailyLogReturn"]=np.log(1+stock_values["Close"].pct_change())
    stock_values["DailyLogReturn"] /= stock_values["DailyLogReturn"].std()
    stock_values.drop(columns=["Open","High","Low","Close","NormClose","Volume"],inplace=True)
    
values=pd.concat(values, keys=stocks.index).dropna()
values.head(n=5)

DailyLogReturn
Symbol Date                                     
AAPL   2019-12-31 00:00:00-05:00        0.365015
       2020-01-02 00:00:00-05:00        1.131198
       2020-01-03 00:00:00-05:00       -0.489863
       2020-01-06 00:00:00-05:00        0.397952
       2020-01-07 00:00:00-05:00       -0.236387

## 3. Correlation

**Reset the index from value dataset and get a pivot matrix**

In [6]:
values=values.reset_index()
DailyReturn=values.pivot(index="Date",columns="Symbol",values="DailyLogReturn").dropna()
DailyReturn.head(n=2)

Symbol,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMD,AMGN,AMT,AMZN,...,TXN,UNH,UNP,UPS,USB,V,VZ,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-31 00:00:00-05:00,0.365015,0.014386,0.042003,-0.017963,0.181035,0.252212,0.225267,0.202338,0.219577,0.022695,...,0.249570,0.023412,0.203874,0.279776,0.084648,0.021234,0.229962,0.151088,-0.330545,0.198953
2020-01-02 00:00:00-05:00,1.131198,0.721103,0.062923,-0.107956,0.563758,0.326203,2.066526,-0.245412,-0.299030,1.182228,...,0.503341,-0.267212,0.468695,-0.121648,-0.063469,0.968263,-0.424179,-0.037722,0.059124,0.735300


**Calculate Correlation matrix**

In [7]:
tickers=DailyReturn.columns
dcor_matrix=pd.DataFrame(index=tickers,columns=tickers,dtype=float)

for i, stock_a in enumerate(tickers):
    for j, stock_b in enumerate(tickers):
        if i<j:
            dcor_value=dcor.distance_correlation(
                DailyReturn[stock_a].values,
                DailyReturn[stock_b].values
            )
            dcor_matrix.loc[stock_a,stock_b]=dcor_value
            dcor_matrix.loc[stock_b,stock_a]=dcor_value

for ticker in tickers:
    dcor_matrix.loc[ticker, ticker]=1.0
    

print (dcor_matrix)

Symbol      AAPL      ABBV       ABT       ACN      ADBE       AIG       AMD  \
Symbol                                                                         
AAPL    1.000000  0.217205  0.372865  0.513727  0.592649  0.273366  0.511462   
ABBV    0.217205  1.000000  0.387878  0.283796  0.208033  0.292431  0.117149   
ABT     0.372865  0.387878  1.000000  0.463536  0.395832  0.253038  0.278010   
ACN     0.513727  0.283796  0.463536  1.000000  0.594553  0.425236  0.421808   
ADBE    0.592649  0.208033  0.395832  0.594553  1.000000  0.236958  0.552434   
...          ...       ...       ...       ...       ...       ...       ...   
V       0.488190  0.319582  0.439751  0.575808  0.506333  0.438106  0.379545   
VZ      0.194868  0.293099  0.320791  0.288447  0.133761  0.291986  0.089661   
WFC     0.265548  0.271463  0.258503  0.392064  0.216596  0.649613  0.218161   
WMT     0.292131  0.253574  0.301785  0.299431  0.250980  0.201942  0.194434   
XOM     0.203059  0.267827  0.152620  0.

In [13]:
dcor_long=dcor_matrix.reset_index()

dcor_long=dcor_long.melt(
    id_vars="Symbol",
    var_name="stock_b",
    value_name="correlation"
)

dcor_long=dcor_long.dropna()
dcor_long=dcor_long[dcor_long["Symbol"] != dcor_long["stock_b"]]
dcor_long=dcor_long.rename(columns={"Symbol":"stock_a"})

dcor_long.to_csv("../data/correlation.csv",index=False)

**Dissimilarity Matrix**

In [43]:
dissimilarity_matrix=1-dcor_matrix
for ticker in tickers:
    dissimilarity_matrix.loc[ticker,ticker]=0.0

print(dissimilarity_matrix)

Symbol      AAPL      ABBV       ABT       ACN      ADBE       AIG       AMD  \
Symbol                                                                         
AAPL    0.000000  0.783416  0.627916  0.486822  0.407688  0.726962  0.488900   
ABBV    0.783416  0.000000  0.612878  0.716952  0.792531  0.707951  0.883222   
ABT     0.627916  0.612878  0.000000  0.537215  0.604718  0.747353  0.722526   
ACN     0.486822  0.716952  0.537215  0.000000  0.405783  0.575019  0.578588   
ADBE    0.407688  0.792531  0.604718  0.405783  0.000000  0.763308  0.447854   
...          ...       ...       ...       ...       ...       ...       ...   
V       0.512382  0.681115  0.561040  0.424690  0.494101  0.562130  0.620923   
VZ      0.805722  0.707519  0.679946  0.712262  0.866760  0.708403  0.910552   
WFC     0.735086  0.729214  0.742285  0.608489  0.783854  0.350553  0.782282   
WMT     0.708768  0.747382  0.699370  0.701663  0.749722  0.798659  0.806204   
XOM     0.797150  0.732389  0.847494  0.

**Aplication of the Minimum Spanning Tree (MST)**

In [44]:
G = nx.Graph()
for i, stock_a in enumerate(tickers):
    for j, stock_b in enumerate(tickers):
        if i<j:
            weight = dissimilarity_matrix.loc[stock_a,stock_b]
            G.add_edge(stock_a,stock_b,weight=weight)

mst=nx.minimum_spanning_tree(G)

print(list(mst.edges(data=True)))

[('AAPL', 'MSFT', {'weight': np.float64(0.31627603970954343)}), ('AAPL', 'TSLA', {'weight': np.float64(0.5397262666954017)}), ('ABBV', 'BMY', {'weight': np.float64(0.5247050067698777)}), ('ABT', 'TMO', {'weight': np.float64(0.41383385927578575)}), ('ABT', 'MDT', {'weight': np.float64(0.47093970917919525)}), ('ACN', 'INTU', {'weight': np.float64(0.3788777848089405)}), ('ACN', 'BLK', {'weight': np.float64(0.40587366295317684)}), ('ACN', 'LIN', {'weight': np.float64(0.42562908211228545)}), ('ACN', 'CSCO', {'weight': np.float64(0.4295295760489998)}), ('ADBE', 'MSFT', {'weight': np.float64(0.28758117191186217)}), ('ADBE', 'INTU', {'weight': np.float64(0.30034762912695356)}), ('ADBE', 'CRM', {'weight': np.float64(0.3282394481162684)}), ('AIG', 'MET', {'weight': np.float64(0.22233016848587328)}), ('AMD', 'NVDA', {'weight': np.float64(0.27975566939056573)}), ('AMGN', 'JNJ', {'weight': np.float64(0.5036435598706811)}), ('AMGN', 'GILD', {'weight': np.float64(0.5117627555234432)}), ('AMT', 'NEE',

In [45]:
mst_edges=[{"stock_a":edge[0],"stock_b":edge[1], "weight":edge[2]["weight"]} for edge in mst.edges(data=True)]
mst_df=pd.DataFrame(mst_edges)
mst_df.to_csv("../data/mst_edges.csv",index=False)